# Day 7: Bridge Repair
```
The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.
The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?
```

In [2]:
import re
import sys
sys.path.append("..")

from common_processing import read_lines, read_as_character_array,\
    read_columns, read_diagonals

test_path = "test.txt"
data_path = "data.txt"

# data = read_lines(test_path)
# data

In [3]:
# Wrapp this transformation into a function for later
def create_data_dict(path: str) -> dict[int:list]:
    """ 
    Let's parse the list as a dictionary of integers to a list of integers
    NOTE: this structure will not work if some results are duplicated ... 
    It looks like there are none in the full data, so we are clear.
    """

    data = read_lines(path)
    data = {
        int(number):components for number, components in 
        [line.split(":") for line in data]
    }
    # Turn the values of the dict into a list of integers
    for key, value in data.items():
        data[key] = [int(x) for x in value.strip().split(" ")]

    return data

data = create_data_dict(test_path)
data

{190: [10, 19],
 3267: [81, 40, 27],
 83: [17, 5],
 156: [15, 6],
 7290: [6, 8, 6, 15],
 161011: [16, 10, 13],
 192: [17, 8, 14],
 21037: [9, 7, 18, 13],
 292: [11, 6, 16, 20]}

In [ ]:
# Approach: 
""" 
For each list, compute each possible permutation of adding and multiplying
if any match the key, then select it.

Later we add all valid values
"""

[81, 40, 27]

In [ ]:
import itertools
def get_operation_options(length: int) -> itertools.product:
    """ 
    For a given lenght, there are length - 1 operation spaces and
    2 possible combinations, add + or multiply *
    length: actual length of the list of numbers

    returns
    """

    operations = ["+", "*"]

    return itertools.product(operations, repeat=(length - 1))

options = get_operation_options(3)
[print(o) for o in options]

('+', '+')
('+', '*')
('*', '+')
('*', '*')


[None, None, None, None]

In [ ]:
# Compute the result of one list and one option for its operations

def compute_operations(numbers: list[int], operations: tuple[str]) -> int:
    """ 
    Given a list of numbers and a list of operations, compute its value.
    """ 
    # the first number is the starting point, then use the rest
    result = numbers[0]
    numbers = numbers[1:]
    for n, o in zip(numbers, operations):

        if o == "+":
            result += n
        if o == "*":
            result *= n

    return result

compute_operations([81, 40, 27], ("*", "+"))  # 3267 : [81, 40, 27]

3267

In [49]:
# Loops
value = 3267
components = data[value]
options = get_operation_options(len(components))

for o in options:
    if compute_operations(components, o) == value:
        print(value)


3267
3267


In [50]:
# loop over all data:
result = 0
for value, components in data.items():
    options = get_operation_options(len(components))
    for o in options:
        if compute_operations(components, o) == value:
            result += value
            break

print(result)

3749


In [51]:
# Try on all data
data = create_data_dict(data_path)

result = 0
for value, components in data.items():
    options = get_operation_options(len(components))
    for o in options:
        if compute_operations(components, o) == value:
            result += value
            break

print(result)

1153997401072


# Part Two ---
```
The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

156: 15 6 can be made true through a single concatenation: 15 || 6 = 156.
7290: 6 8 6 15 can be made true using 6 * 8 || 6 * 15.
192: 17 8 14 can be made true using 17 || 8 + 14.
Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?
```
 

In [61]:
# we just add another symbol and operation to compute.
def get_operation_options_2(length: int) -> itertools.product:
    """ 
    For a given lenght, there are length - 1 operation spaces and
    2 possible combinations, add + or multiply *
    length: actual length of the list of numbers

    returns
    """

    operations = ["+", "*", "||"]

    return itertools.product(operations, repeat=(length - 1))
options = get_operation_options_2(7)
len([o for o in options])

729

In [62]:
def compute_operations_2(numbers: list[int], operations: tuple[str]) -> int:
    """ 
    Given a list of numbers and a list of operations, compute its value.
    """ 
    # the first number is the starting point, then use the rest
    result = numbers[0]
    numbers = numbers[1:]
    for n, o in zip(numbers, operations):

        if o == "+":
            result += n
        if o == "*":
            result *= n
        if o == "||":
            result = int(str(result) + str(n))

    return result

In [63]:
# run with test
data = create_data_dict(test_path)

result = 0
for value, components in data.items():
    options = get_operation_options_2(len(components))
    for o in options:
        if compute_operations_2(components, o) == value:
            result += value
            break

print(result)


11387


In [64]:
# run with data
data = create_data_dict(data_path)

result = 0
for value, components in data.items():
    options = get_operation_options_2(len(components))
    for o in options:
        if compute_operations_2(components, o) == value:
            result += value
            break

print(result)


97902809384118
